# For Windows
1. Please download the latest SasView v5 package from
   https://github.com/SasView/sasview/tags
2. Install SasView
3. Adjust the *sasviewProgramPath* variable accordingly for the path where SasView was installed.

## In case of error
<pre style="background: #fdd">
    CalledProcessError: Command '['gcc', ...
</pre>
Please start the previously installed SasView at least once and create a plot with the desired model function. This will build the selected model from C-sources into a python library in the directory <br />
`C:\Users\{user}\.sasmodels\compiled_models\sas32_{model}.so`

If it still fails, it might be caused by OpenCL being installed while the *pyopencl* package can not be imported in jupyter notebook/lab. Try to disable OpenCL entirely by setting `os.environ["SAS_OPENCL"] = "none"` early in the code and check that the desired SasView model was build already (see above).

# McSAS example

This is the same example as in the McSAS3 notebook, but without the extraeneous explanations and information around it. 

In [ ]:
# import all the necessary bits and bobs

import h5py, sys, os
import numpy as np
import pandas
# import scipy
# import multiprocessing
from pathlib import Path

# load required modules
homedir = os.path.expanduser("~")
# disable OpenCL for multiprocessing on CPU
os.environ["SAS_OPENCL"] = "none"

# CHANGE to location where the SasView/sasmodels are installed
sasviewPath = os.path.join(homedir, "Code", "sasmodels")  # <-- change! 
if sasviewPath not in sys.path:
    sys.path.append(sasviewPath)
# import from this path
import sasmodels
import sasmodels.core
import sasmodels.direct_model

# CHANGE this one to whereever you have mcsas3 installed:
mcsasPath = os.path.join(homedir, "Code", "mcsas3")  # <-- change!
if mcsasPath not in sys.path:
    sys.path.append(mcsasPath)

# import from this path:
from mcsas3 import McHat
from mcsas3 import McData1D, McData2D
from mcsas3.mcmodelhistogrammer import McModelHistogrammer
from mcsas3.mcanalysis import McAnalysis
# optimizeScalingAndBackground: takes care of the calculation of the reduced chi-squared value, after a least-squares optimization for the scaling and background factors.
# McModel: extends the SasModel with information on the parameter set and methods for calculating a total scattering intensity from multiple contributions. It also tracks parameter bounds, random generators and picks.
# McOpt: contains mostly settings related to the optimization process. Also keeps track of the contribution to optimize.
# McCore: Contains the methods required to do the optimization. 

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("ggplot")

In [ ]:
# set a filename for documenting the fit:
resPath = Path("..", "testdata", "quickstartdemo1_fitResult.h5")
# delete if it exists:
if resPath.is_file(): resPath.unlink()
# measurement data:
mds = McData1D.McData1D(
    filename=Path("..", "testdata", "quickstartdemo1.csv"),
    nbins=0, # no rebinning in this example
    dataRange = [0.01, 1], # this clips the data to the specified range

    # arguments for pandas.read_csv:
    csvargs = {"sep" : ';', # field delimiter, for flexible whitespace, use: "\s+|\t+|\s+\t+|\t+\s+" (https://stackoverflow.com/questions/15026698/how-to-make-separator-in-pandas-read-csv-more-flexible-wrt-whitespace-for-irreg#15026839)
               "skipinitialspace" : True, # ignore initial blank spaces
               "skip_blank_lines" : True, # ignore lines with nothing in them
               "skiprows" : 0, # skip this many rows before reading data (useful for PDH, which I think has five (?) header rows?)
               "engine": "python", # most flexible
               "header" : None, # let's not read any column names since they're unlikely to match with our expected column names:
               "names": ["Q", "I", "ISigma"], # our expected column names
               "index_col" : False}, # no index column before every row (who does this anyway?)
)

# store the data and all derivatives in the output file:
mds.store(resPath)

# plot the loaded data
fhs, ahs = plt.subplots(nrows = 1, ncols = 1, figsize = [6, 4])
mds.rawData.plot('Q', 'I', yerr= 'ISigma', ax = ahs, label = 'As provided data')
mds.clippedData.plot('Q', 'I', yerr= 'ISigma', ax = ahs, label = 'clipped data')
mds.binnedData.plot('Q', 'I', yerr= 'ISigma', ax = ahs, label = 'binned data')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Q (1/nm)')
plt.ylabel('I (1/(m sr))')
# plt.xlim([0.1, 3])
print(f'data fed to McSAS3 is {mds.measDataLink}')
md = mds.measData.copy() # here we copy the data we want for fitting.

In [ ]:
# configures the model the Monte Carlo optimizer:

mh = McHat.McHat(
            modelName="sphere", # the model name chosen from the list above
            nContrib=300, # number of contributions, 300 normally suffice
            modelDType="default", # choose "fast" for single-precision calculations at your own risk
            fitParameterLimits={"radius": (3.14, 314)}, # this is the parameter we want to MC optimize within these bounds
            staticParameters={ # these are the parameters we don't want to change:
                "background": 0, # is optimized separately, always set to zero
                "scaling": 1, # ibid.
                "sld": 77.93, # SLD of silver 
                "sld_solvent": 9.45,# SLD of water
                },
            maxIter=100000, # don't try more than this many iterations
            convCrit=1, # convergence criterion, should be 1 if reasonable uncertainty estimates are provided, to prevent over- or under-fitting
            nRep=10, # number of independent repetitions of the optimization procedure. 10 for fast, 50-100 for publications
            nCores=2, # number of threads to spread this over. Set to 0 for automatic detection of maximum number of threads
            seed=None, # random number generator seed. Set to a specific value for reproducible random numbers
        )
mh.store(resPath)

In [ ]:
ans = input("This will run the optimization whic can take a few minutes. Are you sure? Y/N")
if ans=='Y': mh.run(md, resPath)

In [ ]:
# histogram the result, This doesn't take so long and can be repeated as required.
histRanges = pandas.DataFrame(
    [
        dict(
            parameter="radius",  # we only varied one parameter, so not much choice here
            nBin=50,             # number of bins
            binScale="log",      # logarithmically spaced along x
            binWeighting="vol",  # volume weighting (only option as of yet)
            autoRange=True,      # automatically sets min/max to histogram all
        ),
        dict(
            parameter="radius",  # same
            nBin=20,             # but only 20 bins
            binScale="linear",   # linear x-axis spacing
            presetRangeMin=3.14, # minimum
            presetRangeMax=25,   # maximum (only first mode)
            binWeighting="vol",  # volume-weighted
            autoRange=False,     # no auto-ranging (only a subset within the specified parameter range)
        ),
        dict(
            parameter="radius",
            nBin=20,
            binScale="linear",
            presetRangeMin=25,
            presetRangeMax=75,
            binWeighting="vol",
            autoRange=False,
        ),

        dict(
            parameter="radius",
            nBin=20,
            binScale="linear",
            presetRangeMin=75,
            presetRangeMax=150,
            binWeighting="vol",
            autoRange=False,
        ),
    ]
)

mcres = McAnalysis(resPath, md, histRanges, store=True)




In [ ]:
## plot the histogram result in a report similar to the original McSAS
# Set up the plotting frame
fhs, ahs = plt.subplots(
    nrows = 2, 
    ncols = 1+len(histRanges), 
    figsize = [6 * (1+len(histRanges)), 5], 
    gridspec_kw={
        'width_ratios':list(np.ones(len(histRanges) + 1)), 
        'height_ratios':[1,2]
        }
    )
csfont = {'fontname':'Courier New'}


# plot a histogram for every histRange:
for histNum, histRange in histRanges.iterrows():
    histDataFrame = mcres._averagedHistograms[histNum]
    plt.sca(ahs[1, 1 + histNum])
    plt.bar(
        histDataFrame['xMean'], 
        histDataFrame['yMean'], 
        align = 'center', 
        width = histDataFrame['xWidth'],
        yerr = histDataFrame['yStd'],
        facecolor = 'orange',
        edgecolor = 'black',
        ecolor = 'red',
        )
    plt.xscale(histRange.binScale)
    plt.xlabel('Size (nm)')
    plt.ylabel('Volume fraction (arb. units)')

    # get report, some string replacements to prevent errors of "missing Glyph (9), which is the tab"
    histReport = mcres.debugReport(histNum)#.split('\n', 1)[1]
    plt.sca(ahs[0, 1 + histNum]) # top right
    ahs[0,1 + histNum].set_aspect(1)
    ahs[0,1 + histNum].axis('off')
    ahs[0,1 + histNum].text(.2, 0, histReport, **csfont, 
        rotation=0,
        horizontalalignment='center',
        verticalalignment='bottom',
        multialignment='left',
        transform=ahs[0,1 + histNum].transAxes,
        bbox=dict(facecolor='white', alpha=0)
    )


# plot data and fit:
plt.sca(ahs[1, 0])
mds.binnedData.plot('Q', 'I', yerr= 'ISigma', ax = ahs[1,0], label = 'Measured data', zorder = 1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Q (1/nm)')
plt.ylabel('I (1/cm)')
# plt.xlim(1e-1, 2)
plt.plot(mcres._measData['Q'][0], mcres.modelIAvg.modelIMean.values, zorder = 2, label = 'McSAS3 result')
plt.legend()

# plot fitting statistics:
runReport = mcres.debugRunReport().split('\n', 1)[1]
plt.sca(ahs[0, 0])
ahs[0,0].set_aspect(1)
ahs[0,0].axis('off')
ahs[0,0].text(.2, 0, runReport, **csfont, 
    rotation=0,
    horizontalalignment='center',
    verticalalignment='bottom',
    multialignment='left',
    transform=ahs[0,0].transAxes,
    bbox=dict(facecolor='white', alpha=0)
)
plt.tight_layout()


# save into PDF:
plt.savefig(Path(resPath.parent, resPath.stem+'.pdf'))


